# Optimizer benchmarks

Run different convex optimizers with different settings on the japan and north america examples to compare accuracy and performance.

In [ ]:
%config InlineBackend.figure_format = "retina"
# Relevant for the linear solver faer in clarabel:
%env RAYON_NUM_THREADS=4

In [ ]:
import datetime
import itertools

import cvxpy as cp
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import tqdm
from loguru import logger

import celeri
import celeri.optimize

logger.disable("celeri")

In [ ]:
command_files = {
    "japan": "../data/command/japan_command_cmi_coupling.json",
    "north_america": "../data/command/western_north_america_command_coupling.json",
}

# We run through all combinations of these settings
setting_lists = {
    "with_limits": [
        None,
        (-110, 110),
    ],
    "rescale_parameters": [True, False],
    "rescale_constraints": [True, False],
    "objective": celeri.optimize.Objective.__args__,
    "solver": [
        ("CLARABEL", {"equilibrate_enable": False}),
        ("CLARABEL", {"equilibrate_enable": False, "direct_solve_method": "faer"}),
        ("CLARABEL", {"equilibrate_enable": True}),
        ("CUSTOM_CVXOPT", {}),
    ],
}

if "GOROBI" in cp.installed_solvers():
    setting_lists["solver"].append(("GUROBI", {}))

In [ ]:
results = []

for name, path in command_files.items():
    problem = celeri.optimize.build_problem(path)

    combinations = list(itertools.product(*setting_lists.values()))
    for args in tqdm.tqdm(combinations, total=len(combinations)):
        kwargs = dict(zip(setting_lists.keys(), args))
        solver, solve_args = kwargs.pop("solver")
        out = celeri.optimize.benchmark_solve(
            problem=problem,
            solver=solver,
            solve_kwargs=solve_args,
            **kwargs,
        )
        out["solver_path"] = str((solver, solve_args))
        out["config"] = str(
            (
                solver,
                solve_args,
                kwargs["rescale_parameters"],
                kwargs["rescale_constraints"],
            )
        )
        out["dataset"] = name
        results.append(out)

ds = pl.from_dicts(results)
ds = ds.with_columns(
    params=ds["params"].map_elements(
        lambda x: np.array(x), return_dtype=pl.List(pl.Float64)
    ),
    params_raw=ds["params_raw"].map_elements(
        lambda x: np.array(x), return_dtype=pl.List(pl.Float64)
    ),
)
now = datetime.datetime.now(datetime.UTC)
filename = f"{now.isoformat(timespec='seconds')}_benchmark_opt_solver.parquet"
ds.write_parquet(filename)

## Japan

In [ ]:
df = pd.json_normalize(results)

In [ ]:
sns.catplot(
    data=df.assign(limits=lambda x: x.limits.astype(str)).query("dataset == 'japan'"),
    y="time",
    x="solver",
    row="limits",
    hue="config",
    col="objective",
    kind="bar",
)

In [ ]:
sns.catplot(
    data=df.assign(limits=lambda x: x.limits.astype(str)).query("dataset == 'japan'"),
    y="objective_norm2",
    x="solver",
    row="limits",
    hue="config",
    col="objective",
    kind="bar",
)

## North America

In [ ]:
sns.catplot(
    data=df.assign(limits=lambda x: x.limits.astype(str)).query(
        "dataset == 'north_america'"
    ),
    y="time",
    x="solver",
    row="limits",
    hue="config",
    col="objective",
    kind="bar",
)

In [ ]:
sns.catplot(
    data=df.assign(limits=lambda x: x.limits.astype(str)).query(
        "dataset == 'north_america'"
    ),
    y="objective_norm2",
    x="solver",
    row="limits",
    hue="config",
    col="objective",
    kind="bar",
)

## Run full optimization

In [ ]:
%%time
problem = celeri.optimize.build_problem(command_files["japan"])

solve_kwargs = dict(
    solver="CLARABEL",
    equilibrate_enable=False,
    direct_solve_method="faer",
    presolve_enable=False,
    warm_start=True,
    ignore_dpp=False,
)

trace_japan = celeri.optimize.minimize(
    problem,
    verbose=True,
    velocity_upper=110.0,
    velocity_lower=-110.0,
    solve_kwargs=solve_kwargs,
    objective="sum_of_squares",
)

In [ ]:
%%time
problem = celeri.optimize.build_problem(command_files["north_america"])

solve_kwargs = dict(
    solver="CLARABEL",
    equilibrate_enable=False,
    direct_solve_method="faer",
    presolve_enable=False,
    warm_start=True,
    ignore_dpp=False,
)

trace_north_america = celeri.optimize.minimize(
    problem,
    verbose=True,
    velocity_upper=110.0,
    velocity_lower=-110.0,
    solve_kwargs=solve_kwargs,
    objective="sum_of_squares",
)